In [1]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd

In [2]:
from selenium.common.exceptions import ElementClickInterceptedException

In [3]:
# Declare browser
s = Service('C://Users//hantt3//Downloads//chromedriver_win32')
driver = webdriver.Chrome(service=s)
url = 'https://www.google.com/maps/place/Hong+Ngoc+General+Hospital/@21.0424872,105.8416019,17z/data=!4m8!3m7!1s0x3135abba7ba53e11:0x587dbd43c0113ca9!8m2!3d21.0424822!4d105.8441768!9m1!1b1!16s%2Fg%2F1yl48pt04'
driver.get(url)
sleep(3)



In [4]:
#scroll the full page of google review
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    sleep(5)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break 


In [5]:
'''create a dataframe including review name and review content because some review doesn't include review content
    therefore if we create a full dataframe with name, rating, time and review, the dataframe only return review with full 
    these 4 values. For example, Phuong Dong general hosital have more than 500 reviews but the below code only returns
    more than 400 values.
''' 
review_name, review_content = [], [] #create 2 emply lists of review name and review content
for review in driver.find_elements(By.CSS_SELECTOR , ".jJc9Ad "):  #loop for all reviews 
    try:
        button = review.find_elements(By.TAG_NAME, 'button') #find the button to click on more to get the full review content
        for m in button:
            if m.text == "More":
                m.click()
    except ElementClickInterceptedException:
        print("ElementClickInterceptedException")
        continue

    name = review.find_elements(By.CSS_SELECTOR ,".d4r55 ")  #find the name of reviewer
    text = review.find_elements(By.CSS_SELECTOR , ".MyEned .wiI7pd") #find the content of review
    
    for j,p in zip(name,text): #loop through the name and text above and use text method to extract only text 
        review_name.append(j.text)
        review_content.append(p.text)

ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementClickInterceptedException
ElementCli

In [6]:
#create a datafram df0 with name and review
df0 = pd.DataFrame(
    {'name': review_name,
     'review': review_content})

In [7]:
df0.shape #check the shape of the dataframe

(351, 2)

In [8]:
df0.head() #see the first 5 rows 

,name,review
0,LESYA NEBLOGGER,"A doctor's visit to this clinic is 500.000, bu..."
1,Jorja,"As an English-speaking tourist, I had the best..."
2,Digital Nomad,Excellent service for foreigners! Came at 10:0...
3,C Cutler,English speaker maintained continuous contact ...
4,Hendrik Boelck,I was there because had some a cough which did...


In [9]:
'''for the code, I dont't need to click on button more and only need to extract name, rating and duration, and each review 
    all include these 3 info, therefore, I will use list comprehension to parse them to list and convert them to dataframe. 
'''
name = driver.find_elements(By.CSS_SELECTOR ,".d4r55 ")
review_name = [elem.text for elem in name]
rate = driver.find_elements(By.CSS_SELECTOR ,".DU9Pgb .kvMYJc")
rating = [elem.get_attribute("aria-label") for elem in rate]
duration = driver.find_elements(By.CSS_SELECTOR ,".DU9Pgb .rsqaWe")
time = [elem.text for elem in duration]

df1 = pd.DataFrame(list(zip(review_name, rating, time)), 
                   columns = ['name_comment', 'rating','time'])

In [10]:
df1.shape

(589, 3)

In [11]:
df1.head()

,name_comment,rating,time
0,LESYA NEBLOGGER,2 stars,5 months ago
1,Jorja,5 stars,2 months ago
2,Digital Nomad,5 stars,a month ago
3,C Cutler,5 stars,2 years ago
4,Hendrik Boelck,5 stars,a month ago


In [12]:
'''
Left merge on pandas: Keep every row in the left dataframe here df1. 
Where there are missing values of the “on” variable in the right dataframe df0, add empty / NaN values in the result.
''' 
df = df1.merge(df0, how = 'left', left_on = "name_comment", right_on = "name") #merge the df0 to df1 based on name column


'Left merge on pandas: Keep every row in the left dataframe here df1. \nWhere there are missing values of the “on” variable in the right dataframe df0, add empty / NaN values in the result.'

In [13]:
df.shape

(593, 5)

In [14]:
df.duplicated().sum()

0

In [15]:
df.tail()

,name_comment,rating,time,name,review
588,Quynh Anh Hoang,5 stars,3 years ago,NaN,NaN
589,Đỗ Đức Bảo,5 stars,7 months ago,NaN,NaN
590,Huong Nguyenvan,1 star,2 years ago,NaN,NaN
591,Charlotttibbals Geiger,4 stars,4 months ago,NaN,NaN
592,Diệu Nguyễn Minh,5 stars,2 months ago,NaN,NaN


In [16]:
df.drop('name', axis = 1, inplace=True)

In [17]:
df.shape

(593, 4)

In [18]:
df.to_csv('hongngoc_google_reviews_27_4.csv',index=False)
print("Sucessful")

Sucessful
